
### Objective          :  Get 0.69 score  https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/leaderboard and convert to a Keras Tutorial

### Details              :  https://github.com/LamUong/FacialExpressionRecognition
### Current Status :  64.27

### Some observaions

#### the test accuracy and private leader board accuracy are very close

#### after 300 trials there sees to have over fitting

In [38]:
from __future__ import print_function

import cv2
import PIL
from PIL import Image
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.utils import np_utils
#from keras.regularizers import l2, activity_l2
from keras.regularizers import l2, l1
from keras.layers import deserialize as layer_from_config
import numpy as np
import _pickle as cPickle
import numpy
import cv2
import scipy
import csv
import dataprocessing

## Define the model

In [2]:
def model_generate():
    img_rows, img_cols = 48, 48

    model = Sequential()
    model.add(Convolution2D(64, 5, 5, border_mode='valid', input_shape=(img_rows, img_cols, 1)))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(2, 2), dim_ordering='th'))
    model.add(MaxPooling2D(pool_size=(5, 5),strides=(2, 2)))

    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th'))
    model.add(Convolution2D(64, 3, 3))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th')) 
    model.add(Convolution2D(64, 3, 3))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.AveragePooling2D(pool_size=(3, 3),strides=(2, 2)))

    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th'))
    model.add(Convolution2D(128, 3, 3))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th'))
    model.add(Convolution2D(128, 3, 3))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
 
    model.add(keras.layers.convolutional.ZeroPadding2D(padding=(1, 1), dim_ordering='th'))
    model.add(keras.layers.convolutional.AveragePooling2D(pool_size=(3, 3),strides=(2, 2)))
 
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(Dropout(0.2))
    model.add(Dense(1024))
    model.add(keras.layers.advanced_activations.PReLU(init='zero', weights=None))
    model.add(Dropout(0.2))

    model.add(Dense(7))
    model.add(Activation('softmax'))

    ada = Adadelta(lr=0.1, rho=0.95, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy',
    optimizer=ada,
    metrics=['accuracy'])
    model.summary()
    return model

In [3]:
def ConvertTo3DVolume(data):
    img_rows, img_cols = 48, 48
    test_set_x = numpy.asarray(data) 
    test_set_x = test_set_x.reshape(test_set_x.shape[0],img_rows,img_cols)
    test_set_x = test_set_x.reshape(test_set_x.shape[0], 1, img_rows, img_cols)
    test_set_x = test_set_x.astype('float32')
    return test_set_x

In [4]:
def predict_prob(number,test_set_x,model):
    toreturn = []
    for data5 in test_set_x:
        if number ==0:
            toreturn.append(dataprocessing.Flip(data5))
        elif number ==1:
            toreturn.append(dataprocessing.Roated15Left(data5))
        elif number ==2:
            toreturn.append(dataprocessing.Roated15Right(data5))
        elif number ==3:
            toreturn.append(dataprocessing.shiftedUp20(data5))
        elif number ==4:
            toreturn.append(dataprocessing.shiftedDown20(data5))
        elif number ==5:
            toreturn.append(dataprocessing.shiftedLeft20(data5))
        elif number ==6:
            toreturn.append(dataprocessing.shiftedRight20(data5))
        elif number ==7:
            toreturn.append(data5)
        toreturn = ConvertTo3DVolume(toreturn)
        proba = model.predict_proba(toreturn)
        return proba

In [5]:
model = model_generate()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(48, 48, 1..., padding="valid")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `PReLU` call to the Keras 2 API: `PReLU(weights=None, alpha_initializer="zero")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(2, 2), data_format="channels_first")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `ZeroPadding2D` call to the Keras 2 API: `ZeroPadding2D(padding=(1, 1), data_format="channels_first")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # This is added back by Interactiv

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 44, 44, 64)        1664      
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 44, 44, 64)        123904    
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 44, 48, 68)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 22, 68)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 20, 24, 70)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 22, 64)        40384     
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 18, 22, 64)        25344     
__________

### We tried to load the existing model but it is not working
#### The exisitng model used pytorch so we cannot load into our tensor flow as back end, so we need to train our existing model

In [35]:
#filepath='MPM5.hdf5'
#model.load_weights(filepath)

### Load and preprocess the train and validation data

In [7]:
X_train, y_train, X_test, y_test  = dataprocessing.load_data()

In [8]:
from numpy import array

In [9]:
for i in range(len(X_train)):
    X_train[i] = (array(X_train[i])).reshape(48, 48, 1)

In [10]:
type(X_train[0])


numpy.ndarray

In [11]:
X_train= array(X_train)

In [12]:
X_train.shape

(32181, 48, 48, 1)

In [13]:
for i in range(len(X_test)):
    X_test[i] = (array(X_test[i])).reshape(48, 48, 1)

In [14]:
X_test= array(X_test)

In [15]:
X_test.shape

(3589, 48, 48, 1)

In [16]:
y_train = array(y_train)
y_test = array(y_test)

In [17]:
X_train = np.append(X_train, X_train,axis=0)
y_train = np.append(y_train, y_train,axis=0)

In [18]:
print(y_train.shape, y_test.shape)

(64362,) (3589,)


In [19]:
# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_train = np_utils.to_categorical(y_train, 7)
Y_test = np_utils.to_categorical(y_test, 7)

In [20]:
Y_train[:10]


array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.]], dtype=float32)

In [21]:
#Callback for saving the best model
from keras.callbacks import ModelCheckpoint
filepath= "FaceExp_Reco_tr_02.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [22]:
batch_size = 128
nb_classes = 7
epochs = 1200
img_channels = 1

### Data Augmentation

In [23]:
#Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=40,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,    # randomly flip images
        fill_mode='nearest')

### Train the model

In [24]:
'''model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_data=(X_test, Y_test),
          callbacks=callbacks_list)'''

model.fit_generator(datagen.flow(X_train, Y_train,
          batch_size=batch_size),
          epochs=300,
          verbose=1,
          validation_data=(X_test, Y_test),
          callbacks=callbacks_list)

Epoch 1/300
503/503 [==============================] - 112s 222ms/step - loss: 1.8234 - acc: 0.2498 - val_loss: 1.8118 - val_acc: 0.2449

Epoch 00001: val_acc improved from -inf to 0.24492, saving model to FaceExp_Reco_tr_02.best.hdf5
Epoch 2/300
503/503 [==============================] - 41s 82ms/step - loss: 1.8074 - acc: 0.2526 - val_loss: 1.7958 - val_acc: 0.2561

Epoch 00002: val_acc improved from 0.24492 to 0.25606, saving model to FaceExp_Reco_tr_02.best.hdf5
Epoch 3/300
503/503 [==============================] - 41s 82ms/step - loss: 1.8016 - acc: 0.2541 - val_loss: 1.7918 - val_acc: 0.2536

Epoch 00003: val_acc did not improve from 0.25606
Epoch 4/300
503/503 [==============================] - 41s 82ms/step - loss: 1.7981 - acc: 0.2571 - val_loss: 1.7760 - val_acc: 0.2756

Epoch 00004: val_acc improved from 0.25606 to 0.27556, saving model to FaceExp_Reco_tr_02.best.hdf5
Epoch 5/300
503/503 [==============================] - 41s 82ms/step - loss: 1.7937 - acc: 0.2614 - val_los

Epoch 40/300
503/503 [==============================] - 41s 82ms/step - loss: 1.4381 - acc: 0.4466 - val_loss: 1.3811 - val_acc: 0.4767

Epoch 00040: val_acc did not improve from 0.48398
Epoch 41/300
503/503 [==============================] - 41s 82ms/step - loss: 1.4348 - acc: 0.4491 - val_loss: 1.3205 - val_acc: 0.4862

Epoch 00041: val_acc improved from 0.48398 to 0.48621, saving model to FaceExp_Reco_tr_02.best.hdf5
Epoch 42/300
503/503 [==============================] - 42s 83ms/step - loss: 1.4250 - acc: 0.4523 - val_loss: 1.3279 - val_acc: 0.4848

Epoch 00042: val_acc did not improve from 0.48621
Epoch 43/300
503/503 [==============================] - 41s 82ms/step - loss: 1.4220 - acc: 0.4530 - val_loss: 1.3511 - val_acc: 0.4759

Epoch 00043: val_acc did not improve from 0.48621
Epoch 44/300
503/503 [==============================] - 41s 82ms/step - loss: 1.4115 - acc: 0.4575 - val_loss: 1.3030 - val_acc: 0.4979

Epoch 00044: val_acc improved from 0.48621 to 0.49791, saving mod

503/503 [==============================] - 41s 82ms/step - loss: 1.2600 - acc: 0.5199 - val_loss: 1.1856 - val_acc: 0.5425

Epoch 00080: val_acc did not improve from 0.55336
Epoch 81/300
503/503 [==============================] - 41s 82ms/step - loss: 1.2558 - acc: 0.5225 - val_loss: 1.1784 - val_acc: 0.5444

Epoch 00081: val_acc did not improve from 0.55336
Epoch 82/300
503/503 [==============================] - 41s 82ms/step - loss: 1.2544 - acc: 0.5238 - val_loss: 1.1512 - val_acc: 0.5578

Epoch 00082: val_acc improved from 0.55336 to 0.55782, saving model to FaceExp_Reco_tr_02.best.hdf5
Epoch 83/300
503/503 [==============================] - 41s 82ms/step - loss: 1.2437 - acc: 0.5257 - val_loss: 1.1684 - val_acc: 0.5495

Epoch 00083: val_acc did not improve from 0.55782
Epoch 84/300
503/503 [==============================] - 41s 82ms/step - loss: 1.2453 - acc: 0.5256 - val_loss: 1.1771 - val_acc: 0.5481

Epoch 00084: val_acc did not improve from 0.55782
Epoch 85/300
503/503 [======

503/503 [==============================] - 41s 82ms/step - loss: 1.1487 - acc: 0.5635 - val_loss: 1.1005 - val_acc: 0.5768

Epoch 00121: val_acc did not improve from 0.58930
Epoch 122/300
503/503 [==============================] - 41s 82ms/step - loss: 1.1506 - acc: 0.5644 - val_loss: 1.1075 - val_acc: 0.5795

Epoch 00122: val_acc did not improve from 0.58930
Epoch 123/300
503/503 [==============================] - 41s 82ms/step - loss: 1.1478 - acc: 0.5655 - val_loss: 1.0867 - val_acc: 0.5770

Epoch 00123: val_acc did not improve from 0.58930
Epoch 124/300
503/503 [==============================] - 41s 82ms/step - loss: 1.1431 - acc: 0.5668 - val_loss: 1.1283 - val_acc: 0.5743

Epoch 00124: val_acc did not improve from 0.58930
Epoch 125/300
503/503 [==============================] - 41s 82ms/step - loss: 1.1439 - acc: 0.5670 - val_loss: 1.1032 - val_acc: 0.5773

Epoch 00125: val_acc did not improve from 0.58930
Epoch 126/300
503/503 [==============================] - 41s 82ms/step - l


Epoch 00162: val_acc did not improve from 0.61410
Epoch 163/300
503/503 [==============================] - 42s 83ms/step - loss: 1.0781 - acc: 0.5931 - val_loss: 1.0374 - val_acc: 0.6133

Epoch 00163: val_acc did not improve from 0.61410
Epoch 164/300
503/503 [==============================] - 42s 83ms/step - loss: 1.0799 - acc: 0.5916 - val_loss: 1.0595 - val_acc: 0.6041

Epoch 00164: val_acc did not improve from 0.61410
Epoch 165/300
503/503 [==============================] - 41s 82ms/step - loss: 1.0737 - acc: 0.5953 - val_loss: 1.0486 - val_acc: 0.6077

Epoch 00165: val_acc did not improve from 0.61410
Epoch 166/300
503/503 [==============================] - 41s 82ms/step - loss: 1.0764 - acc: 0.5938 - val_loss: 1.0446 - val_acc: 0.6024

Epoch 00166: val_acc did not improve from 0.61410
Epoch 167/300
503/503 [==============================] - 41s 82ms/step - loss: 1.0720 - acc: 0.5959 - val_loss: 1.0457 - val_acc: 0.6077

Epoch 00167: val_acc did not improve from 0.61410
Epoch 168

503/503 [==============================] - 41s 82ms/step - loss: 1.0248 - acc: 0.6147 - val_loss: 1.0278 - val_acc: 0.6138

Epoch 00206: val_acc did not improve from 0.62106
Epoch 207/300
503/503 [==============================] - 41s 82ms/step - loss: 1.0224 - acc: 0.6180 - val_loss: 1.0172 - val_acc: 0.6233

Epoch 00207: val_acc improved from 0.62106 to 0.62329, saving model to FaceExp_Reco_tr_02.best.hdf5
Epoch 208/300
503/503 [==============================] - 41s 82ms/step - loss: 1.0231 - acc: 0.6138 - val_loss: 1.0105 - val_acc: 0.6216

Epoch 00208: val_acc did not improve from 0.62329
Epoch 209/300
503/503 [==============================] - 41s 82ms/step - loss: 1.0209 - acc: 0.6156 - val_loss: 1.0244 - val_acc: 0.6133

Epoch 00209: val_acc did not improve from 0.62329
Epoch 210/300
503/503 [==============================] - 41s 82ms/step - loss: 1.0173 - acc: 0.6150 - val_loss: 1.0297 - val_acc: 0.6219

Epoch 00210: val_acc did not improve from 0.62329
Epoch 211/300
503/503 [=


Epoch 00248: val_acc did not improve from 0.63165
Epoch 249/300
503/503 [==============================] - 41s 82ms/step - loss: 0.9801 - acc: 0.6294 - val_loss: 1.0022 - val_acc: 0.6269

Epoch 00249: val_acc did not improve from 0.63165
Epoch 250/300
503/503 [==============================] - 41s 82ms/step - loss: 0.9790 - acc: 0.6323 - val_loss: 1.0002 - val_acc: 0.6264

Epoch 00250: val_acc did not improve from 0.63165
Epoch 251/300
503/503 [==============================] - 42s 83ms/step - loss: 0.9786 - acc: 0.6328 - val_loss: 1.0066 - val_acc: 0.6266

Epoch 00251: val_acc did not improve from 0.63165
Epoch 252/300
503/503 [==============================] - 42s 83ms/step - loss: 0.9757 - acc: 0.6344 - val_loss: 1.0160 - val_acc: 0.6180

Epoch 00252: val_acc did not improve from 0.63165
Epoch 253/300
503/503 [==============================] - 41s 82ms/step - loss: 0.9753 - acc: 0.6329 - val_loss: 1.0195 - val_acc: 0.6216

Epoch 00253: val_acc did not improve from 0.63165
Epoch 254

503/503 [==============================] - 41s 82ms/step - loss: 0.9399 - acc: 0.6486 - val_loss: 1.0017 - val_acc: 0.6344

Epoch 00292: val_acc did not improve from 0.63834
Epoch 293/300
503/503 [==============================] - 41s 82ms/step - loss: 0.9397 - acc: 0.6487 - val_loss: 1.0046 - val_acc: 0.6278

Epoch 00293: val_acc did not improve from 0.63834
Epoch 294/300
503/503 [==============================] - 41s 82ms/step - loss: 0.9406 - acc: 0.6473 - val_loss: 1.0018 - val_acc: 0.6317

Epoch 00294: val_acc did not improve from 0.63834
Epoch 295/300
503/503 [==============================] - 41s 82ms/step - loss: 0.9372 - acc: 0.6490 - val_loss: 1.0139 - val_acc: 0.6350

Epoch 00295: val_acc did not improve from 0.63834
Epoch 296/300
503/503 [==============================] - 41s 82ms/step - loss: 0.9359 - acc: 0.6495 - val_loss: 0.9973 - val_acc: 0.6342

Epoch 00296: val_acc did not improve from 0.63834
Epoch 297/300
503/503 [==============================] - 41s 82ms/step - l


## We have done enough training. It is not good enough. However, let's check how the model performs on Private leader board data.

#### Load and process Private leader board data

In [25]:
X_private_test, y_private_test  = dataprocessing.load_test_data()

In [26]:
for i in range(len(X_private_test)):
    X_private_test[i] = (array(X_private_test[i])).reshape(48, 48, 1)

In [28]:
X_private_test= array(X_private_test)
X_private_test.shape

(3589, 48, 48, 1)

In [29]:
# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_private_test = np_utils.to_categorical(y_private_test, 7)

## Evaluate your model on Private Leader board data

In [31]:
model.evaluate(X_private_test, Y_private_test)

3589/3589 [==============================] - 1s 286us/step


[1.008300996716888, 0.6288659794146585]

In [33]:
from keras.models import load_model

new_model = load_model("FaceExp_Reco_tr_02.best.hdf5")
score = new_model.evaluate(X_private_test, Y_private_test, verbose=1)
print('Private Leader Board Test loss:', score[0])
print('Private Leader Board Test accuracy:', score[1])

3589/3589 [==============================] - 1s 312us/step
Private Leader Board Test loss: 0.9854776446750882
Private Leader Board Test accuracy: 0.6383393703313484


## Let's Train the model for a few more epochs and see if things improve

In [34]:
model.fit_generator(datagen.flow(X_train, Y_train,
          batch_size=batch_size),
          epochs=50,
          verbose=1,
          validation_data=(X_test, Y_test),
          callbacks=callbacks_list)

Epoch 1/50
503/503 [==============================] - 42s 83ms/step - loss: 0.9307 - acc: 0.6494 - val_loss: 1.0184 - val_acc: 0.6239

Epoch 00001: val_acc did not improve from 0.63834
Epoch 2/50
503/503 [==============================] - 41s 82ms/step - loss: 0.9335 - acc: 0.6478 - val_loss: 1.0246 - val_acc: 0.6261

Epoch 00002: val_acc did not improve from 0.63834
Epoch 3/50
503/503 [==============================] - 41s 82ms/step - loss: 0.9356 - acc: 0.6462 - val_loss: 1.0012 - val_acc: 0.6330

Epoch 00003: val_acc did not improve from 0.63834
Epoch 4/50
503/503 [==============================] - 41s 82ms/step - loss: 0.9337 - acc: 0.6478 - val_loss: 1.0022 - val_acc: 0.6344

Epoch 00004: val_acc did not improve from 0.63834
Epoch 5/50
503/503 [==============================] - 41s 82ms/step - loss: 0.9292 - acc: 0.6527 - val_loss: 1.0028 - val_acc: 0.6294

Epoch 00005: val_acc did not improve from 0.63834
Epoch 6/50
503/503 [==============================] - 42s 83ms/step - loss:

503/503 [==============================] - 41s 82ms/step - loss: 0.9000 - acc: 0.6622 - val_loss: 1.0093 - val_acc: 0.6322

Epoch 00045: val_acc did not improve from 0.64280
Epoch 46/50
503/503 [==============================] - 41s 82ms/step - loss: 0.8960 - acc: 0.6639 - val_loss: 1.0042 - val_acc: 0.6406

Epoch 00046: val_acc did not improve from 0.64280
Epoch 47/50
503/503 [==============================] - 41s 82ms/step - loss: 0.8982 - acc: 0.6635 - val_loss: 1.0009 - val_acc: 0.6408

Epoch 00047: val_acc did not improve from 0.64280
Epoch 48/50
503/503 [==============================] - 41s 82ms/step - loss: 0.8947 - acc: 0.6669 - val_loss: 1.0379 - val_acc: 0.6311

Epoch 00048: val_acc did not improve from 0.64280
Epoch 49/50
503/503 [==============================] - 41s 82ms/step - loss: 0.8996 - acc: 0.6630 - val_loss: 1.0130 - val_acc: 0.6247

Epoch 00049: val_acc did not improve from 0.64280
Epoch 50/50
503/503 [==============================] - 41s 82ms/step - loss: 0.895

### Evaluate the model again

In [36]:
score = model.evaluate(X_private_test, Y_private_test, verbose=1)
print('Private Leader Board Test loss:', score[0])
print('Private Leader Board Test accuracy:', score[1])

3589/3589 [==============================] - 1s 295us/step
Private Leader Board Test loss: 1.0030053753678796
Private Leader Board Test accuracy: 0.6310950125715267


In [37]:
from keras.models import load_model

new_model = load_model("FaceExp_Reco_tr_02.best.hdf5")
score = new_model.evaluate(X_private_test, Y_private_test, verbose=1)
print('Private Leader Board Test loss:', score[0])
print('Private Leader Board Test accuracy:', score[1])

3589/3589 [==============================] - 1s 322us/step
Private Leader Board Test loss: 1.0010537806411546
Private Leader Board Test accuracy: 0.6427974366450848


### Let's Train some more

In [39]:
model.fit_generator(datagen.flow(X_train, Y_train,
          batch_size=batch_size),
          epochs=50,
          verbose=1,
          validation_data=(X_test, Y_test),
          callbacks=callbacks_list)

Epoch 1/50
503/503 [==============================] - 42s 83ms/step - loss: 0.8977 - acc: 0.6648 - val_loss: 1.0011 - val_acc: 0.6408

Epoch 00001: val_acc did not improve from 0.64280
Epoch 2/50
503/503 [==============================] - 42s 83ms/step - loss: 0.8962 - acc: 0.6637 - val_loss: 1.0271 - val_acc: 0.6317

Epoch 00002: val_acc did not improve from 0.64280
Epoch 3/50
503/503 [==============================] - 42s 83ms/step - loss: 0.8927 - acc: 0.6657 - val_loss: 1.0402 - val_acc: 0.6305

Epoch 00003: val_acc did not improve from 0.64280
Epoch 4/50
503/503 [==============================] - 42s 83ms/step - loss: 0.8898 - acc: 0.6689 - val_loss: 1.0175 - val_acc: 0.6303

Epoch 00004: val_acc did not improve from 0.64280
Epoch 5/50
503/503 [==============================] - 42s 83ms/step - loss: 0.8984 - acc: 0.6635 - val_loss: 1.0075 - val_acc: 0.6383

Epoch 00005: val_acc did not improve from 0.64280
Epoch 6/50
503/503 [==============================] - 42s 83ms/step - loss:

503/503 [==============================] - 42s 83ms/step - loss: 0.8629 - acc: 0.6771 - val_loss: 1.0240 - val_acc: 0.6364

Epoch 00045: val_acc did not improve from 0.64447
Epoch 46/50
503/503 [==============================] - 42s 83ms/step - loss: 0.8650 - acc: 0.6783 - val_loss: 1.0162 - val_acc: 0.6386

Epoch 00046: val_acc did not improve from 0.64447
Epoch 47/50
503/503 [==============================] - 42s 83ms/step - loss: 0.8647 - acc: 0.6758 - val_loss: 1.0081 - val_acc: 0.6353

Epoch 00047: val_acc did not improve from 0.64447
Epoch 48/50
503/503 [==============================] - 42s 83ms/step - loss: 0.8623 - acc: 0.6775 - val_loss: 1.0512 - val_acc: 0.6339

Epoch 00048: val_acc did not improve from 0.64447
Epoch 49/50
503/503 [==============================] - 42s 83ms/step - loss: 0.8598 - acc: 0.6817 - val_loss: 1.0367 - val_acc: 0.6311

Epoch 00049: val_acc did not improve from 0.64447
Epoch 50/50
503/503 [==============================] - 42s 83ms/step - loss: 0.857

In [40]:
score = model.evaluate(X_private_test, Y_private_test, verbose=1)
print('Private Leader Board Test loss:', score[0])
print('Private Leader Board Test accuracy:', score[1])


3589/3589 [==============================] - 1s 285us/step
Private Leader Board Test loss: 1.0101975353357353
Private Leader Board Test accuracy: 0.6425188075004763


In [41]:
from keras.models import load_model

new_model = load_model("FaceExp_Reco_tr_02.best.hdf5")
score = new_model.evaluate(X_private_test, Y_private_test, verbose=1)
print('Private Leader Board Test loss:', score[0])
print('Private Leader Board Test accuracy:', score[1])

3589/3589 [==============================] - 1s 353us/step
Private Leader Board Test loss: 0.9955075966444943
Private Leader Board Test accuracy: 0.644469211512736


#### Looks like it is overfitting